# df_brd_w_prd_v2에서 유실값 확인

In [1]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm import trange, tqdm_notebook
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [2]:
df_brd_w_prd_v1_drop_lg = pd.read_csv("df_brd_w_prd_v1_drop_lg.csv")
df_brd_w_prd_v1_drop_lg = df_brd_w_prd_v1_drop_lg.iloc[:, 1:]

In [3]:
def get_search_len(brd_name):

    try: 
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        print(userAgent)

        options.add_argument('headless')
        options.add_argument('window-size=1920x1080')
        options.add_argument("disable-gpu")
        options.add_argument(f'user-agent={userAgent}')
        wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

        socket.setdefaulttimeout(30)

        url = 'https:/www.glowpick.com/searches/keywords'
        wd.get(url)
        time.sleep(5)
        
        html = wd.page_source
        soup = BeautifulSoup(html, "html.parser")

        wd.find_element_by_xpath("//*[@id='default-layout']/div/div[1]/span/div/div[2]/div[2]/button[2]").click()
        wd.maximize_window()
        time.sleep(3)

        brd_input = wd.find_element_by_xpath("//*[@id='default-layout']/div/div[2]/div/div/div/input")
        brd_input.send_keys(brd_name)
        brd_input.send_keys(Keys.ENTER)
        time.sleep(3)
        
        # ====

        #.products__product
        #.product
        # ====

        def parsing_html_with_class(parsing_info = '.products'):
            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")
            interm_r = soup.select(parsing_info)
            return interm_r
        
        try:
            while True:

                product_lst = parsing_html_with_class()
                
                global product_len
                product_len = len(re.findall('<div class="products__product product" (.*?)', str(product_lst[0])))
                

                element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(product_len - 1))
                wd.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(2)

                product_lst_2 = parsing_html_with_class()

                if product_lst == product_lst_2:
                    html = wd.page_source
                    soup = BeautifulSoup(html, "html.parser")
                    prd_len = len(soup.find_all('div','products__product product'))
                    return prd_len
                    break


        except NoSuchElementException:
            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")
            prd_len = len(soup.find_all('div','products__product product'))
            return prd_len
        
    except Exception as e:
        exception.append(brd_name)
        print("brand name: {}".format(brd_name), e)
        pass

In [5]:
exception = []

In [6]:
df_brd_w_prd_v1_drop_lg_0_10 = df_brd_w_prd_v1_drop_lg.loc[0:10]

In [7]:
for i in tqdm(list(df_brd_w_prd_v1_drop_lg_0_10.index)):
    brd_name = df_brd_w_prd_v1_drop_lg_0_10.loc[i, 'brand_name']
    prd_len = get_search_len(brd_name)
    df_brd_w_prd_v1_drop_lg_0_10.loc[i, 'prd_len'] = prd_len

  0%|          | 0/11 [00:00<?, ?it/s]



====== WebDriver manager ======


Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.3319.102 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Get LATEST driver version for 93.0.4577
Trying to download new driver from https://chromedriver.storage.googleapis.com/93.0.4577.63/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63]


====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


...

Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.2 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======


Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [16]:
df_brd_w_prd_v1_drop_lg_1000_1999

,brand_code_x,brand_name,product_link,product_name,brand_code_y,category,crawling_prd_cnt
1000,https://www.glowpick.com/brands/2607,드레스너에센스,['https://www.glowpick.com/products/41067'],['웰니스 배쓰파우더 - 피어니'],https://www.glowpick.com/brands/2607,['배쓰&바디'],1.0
1001,https://www.glowpick.com/brands/8419,드레스킨,"['https://www.glowpick.com/products/109599', '...","['새틴 틴트 앤 밤', '실크 팩트 밤 [SPF50+/PA+++]', '아이라이터...",https://www.glowpick.com/brands/8419,"['페이스메이크업', '립메이크업', '아이메이크업']",6.0
1002,https://www.glowpick.com/brands/6948,드레싱82,"['https://www.glowpick.com/products/91361', 'h...","['클리어 밀크필 거즈 패드', '마일드필 히알루로닉 겔크림']",https://www.glowpick.com/brands/6948,['스킨케어'],2.0
1003,https://www.glowpick.com/brands/10105,드레싱테이블,['https://www.glowpick.com/products/130467'],['스킨처음 다리미스틱'],https://www.glowpick.com/brands/10105,['스킨케어'],1.0
1004,https://www.glowpick.com/brands/11384,드레아,"['https://www.glowpick.com/products/142949', '...","['히비스커스 엑토인 시크릿 앰플', '[단종] 메이크업 리무버 패드 울트라 퀵'...",https://www.glowpick.com/brands/11384,['스킨케어'],4.0
...,...,...,...,...,...,...,...
1995,https://www.glowpick.com/brands/9695,메그,"['https://www.glowpick.com/products/124383', '...","['투스텝 젤리 마스크 [그린]', '투스텝 젤리 마스크 [퍼플]', '투스텝 젤리...",https://www.glowpick.com/brands/9695,['마스크/팩'],23.0
1996,https://www.glowpick.com/brands/3725,메나드,['https://www.glowpick.com/products/52481'],['원터치 립스틱'],https://www.glowpick.com/brands/3725,['립메이크업'],1.0
1997,https://www.glowpick.com/brands/10846,메노킨,"['https://www.glowpick.com/products/138938', '...","['펩티코튼 크림', '펩티코튼 앰플에센스', '펩티코튼 클렌저']",https://www.glowpick.com/brands/10846,"['스킨케어', '클렌징']",3.0
1998,https://www.glowpick.com/brands/7386,메니콘,"['https://www.glowpick.com/products/97382', 'h...","['메니케어 플러스', '프로젠트']",https://www.glowpick.com/brands/7386,['렌즈'],2.0
